Здравствуйте! В данной лабораторной работе Вы:
* познакомитесь с Audacity;
* используете FFT на аудио данных;
* примените свои уменя анализа данных.

В робототехнике есть ряд задач, в которых идёт работа со звуком. Поэтому, могут пригодиться инструменты для работы со звуком. Хотелось бы отметить [Audacity](https://www.audacityteam.org/) - свободный кросс-платформенный аудио редактор. У него достаточно много возможностей, среди которых отметим полезные для данной работы:
* построение спектра выделенного участка ([документация](https://manual.audacityteam.org/man/plot_spectrum.html));
* отображение спектрограммы ([документация](https://manual.audacityteam.org/man/spectrogram_view.html)).

Демонстрация этих возможностей предоставлена в видеофайле `lab_3b_audacity.mp4`.

Теперь перейдём к заданию. Вам нужно реализовать функцию для анализа аудио данных (WAV файл длиной 15 секунд, 1 канал, 48 КГц, 16 бит со знаком), которая выдаёт двоичный NumPy массив (150 значений False/True, одно значение соответствует 0.1 секунде входного звука).

Начнём со знакомых Вам импортов:

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

Далее немного вспомогательного кода:

In [ ]:
import IPython.display
import base64
import io
import urllib.request
import wave
from json import dumps, loads

def download(url):
  return urllib.request.urlopen(url).read()

def display_player(data):
  IPython.display.display(IPython.display.Audio(data))

def display_link(data, filename="data.wav"):
  html_template = '<a download="{filename}" href="data:audio/wav;base64,{data}" target="_blank">Download</a>'
  html = html_template.format(filename=filename, data=base64.b64encode(data).decode())
  IPython.display.display(IPython.display.HTML(html))

def wave_to_array(data):
  reader = wave.Wave_read(io.BytesIO(data))
  frames = reader.readframes(reader.getnframes())
  return np.frombuffer(frames, dtype=np.int16).astype(np.float32) / np.iinfo(np.int16).max

def array_to_wave(data):
  buffer = io.BytesIO()
  with wave.Wave_write(buffer) as writer:
    writer.setnchannels(1)
    writer.setsampwidth(2)
    writer.setframerate(48000)
    writer.writeframes(((data * np.iinfo(np.int16).max).astype(np.int16)).tobytes())
  return buffer.getvalue()

server = "someserver.com"
task_id = 11

def get_example():
  response = urllib.request.urlopen(server + "/task/" + str(task_id) + "?secret=" + secret)
  token = response.read().decode("utf-8")
  response = urllib.request.urlopen(server + "/data/example_data.wav?case=0&token=" + token)
  example_data = response.read()
  response = urllib.request.urlopen(server + "/data/example_result?case=0&token=" + token)
  example_result = np.array(loads(response.read().decode("utf-8")), dtype=bool)
  return example_data, example_result

def auto_test(func):
  response = urllib.request.urlopen(server + "/task/" + str(task_id) + "?secret=" + secret)
  token = response.read().decode("utf-8")
  results = []
  for case in range(5):
    response = urllib.request.urlopen(server + "/data/data.wav?case=" + str(case) + "&token=" + token)
    data_wave = response.read()
    result = func(data_wave)
    results.append(result.tolist())
  req = urllib.request.Request(server + "/solution?token=" + token)
  req.add_header("Content-Type", "application/json")
  data = dumps(results).encode('utf-8')
  req.add_header('Content-Length', len(data))
  response = urllib.request.urlopen(req, data)
  print("Passed! First time passed:", response.read().decode("utf-8"))

Некоторые из этих функций Вам пригодятся, в том числе:
* `display_link(data, filename="data.wav")` - для данных (`data`) в формате wave выводит ссылку для скачивания wav файла (с именем `filename`)
* `wave_to_array(data)` - для данных (`data`) в формате wave возвращает NumPy массив float чисел в диапазоне от -1 до 1.

Загрузим пример входных и выходных данных.

In [ ]:
example_data, example_result = get_example()

Ознакомимся с входными данными (получим ссылку на загрузку и посмотрим в Audacity).

In [ ]:
display_link(example_data, filename="example_data.wav")

А теперь ознакомимся с выходныими данными.

In [ ]:
example_result

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True, False,
        True, False, False, False, False, False, False,  True, False,
        True, False,  True, False, False, False,  True, False,  True,
       False,  True, False, False, False,  True, False, False, False,
        True, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False,  True,  True,
       False, False,  True,  True, False, False, False,  True, False,
       False, False,  True,  True, False, False,  True, False,  True,
       False,  True, False,  True,  True, False, False, False, False,
       False, False, False,  True,  True, False, False, False, False,
       False, False,  True,  True, False,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

Задание описано в первой текстовой ячейке. Чтобы понять, какая зависимость выходных данных от входных, Вам необходимо внимательно изучить эти данные.

Если у Вас возникли трудности при анализе данных, Вам доступны следующие три подсказки (без каких-либо последствий):

In [ ]:
print(base64.b64decode(b'0JIg0LDRg9C00LjQviDRhNCw0LnQu9C1INC/0YDQuNGB0YPRgtGB0YLQstGD0Y7RgiDQtNCw0L3QvdGL0LUg0L3QsCDQtNCy0YPRhSDQvdC10YHRg9GJ0LjRhSDRh9Cw0YHRgtC+0YLQsNGFLg==').decode('utf-8'))
print(base64.b64decode(b'0J7QtNC90LAg0L3QtdGB0YPRidCw0Y8g0YfQsNGB0YLQvtGC0LAg0L3QsNGF0L7QtNC40YLRgdGPINCyINC00LjQsNC/0LDQt9C+0L3QtSA1MDAtNzAwINCT0YYsINCy0YLQvtGA0LDRjyAtINC+0YIgMTgg0JrQk9GGLg==').decode('utf-8'))
print(base64.b64decode(b'0J3QsCDQutCw0LbQtNC+0Lkg0L3QtdGB0YPRidC10Lkg0YfQsNGB0YLQvtGC0LUg0LXRgdGC0Ywg0LTQstC+0LjRh9C90YvQuSDRgdC40LPQvdCw0LssINC40YLQvtCz0L7QstGL0Lkg0YHQuNCz0L3QsNC7INC30LDQtNCw0L0g0LrQsNC6INC70L7Qs9C40YfQtdGB0LrQsNGPINGE0YPQvdC60YbQuNGPINC00LLRg9GFINC/0LXRgNC10LzQtdC90L3Ri9GFINGN0YLQuNGFINGB0LjQs9C90LDQu9C+0LIu').decode('utf-8'))

В аудио файле присутствуют данные на двух несущих частотах.
Одна несущая частота находится в диапазоне 500-700 Гц, вторая - от 18 КГц.
На каждой несущей частоте есть двоичный сигнал, итоговый сигнал задан как логическая функция двух переменных этих сигналов.


Далее реализуйте функцию анализа данных согласно заданию.

In [ ]:
def analyze(data_wave):
  chunks_amount = 150
  lower_freq = 550
  higher_freq = 20000
  sample_rate = 48000

  # Saving data as array
  data = wave_to_array(data_wave)
  # Data values per discretization time interval. This will be equal to 2*(fft length)
  step = int(data.size / chunks_amount)
  # Initializing result array with False values
  res = np.full(chunks_amount, False, dtype=bool)

  # Calculating fft indices
  higher_indx = int(higher_freq * step / sample_rate)
  lower_indx = int(lower_freq * step / sample_rate)

  # For amplitudes storing
  amps_h = np.full(chunks_amount, 0.0, dtype=float)
  amps_l = np.full(chunks_amount, 0.0, dtype=float)

  # Getting amplitudes
  for i in range(chunks_amount):
    # Processing each slice via fft
    try:
      fft = np.fft.rfft(data[(i * step):((i + 1) * step)])
    except IndexError as err:
      fft = np.fft.rfft(data[(i * step):data.size])

    amps_h[i] = np.abs(fft[higher_indx]) * 2 / data.size
    amps_l[i] = np.abs(fft[lower_indx]) * 2 / data.size

  for i in range(chunks_amount):
    # not a and b -> b > a
    if amps_h[i] < np.mean(amps_h) and amps_l[i] > np.mean(amps_l):
      res[i] = True

  return res

Теперь проверьте, что Ваша функция выдаёт верный результат на примере.

In [ ]:
(example_result == analyze(example_data)).all()

True

Если всё хорошо, попробуйте пройти автоматическую проверку.

In [ ]:
auto_test(analyze)

Passed! First time passed: "2023-05-23T13:17:39.187095"
